In [5]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import UnitaryGate
# Corrected import for random_unitary
from qiskit.quantum_info import random_unitary 
# from qiskit.primitives import Sampler
from qiskit.visualization import circuit_drawer

# Example usage (to verify the fix)
# Generate a random 2-qubit unitary
# u = random_unitary(4) 
# print("Random Unitary generated successfully.")

def create_learned_piece(num_qubits, label):
    """
    Giả lập một 'mảnh' mạch đã được học cục bộ (Learned Local Piece).
    Trong thực tế, đây là kết quả của quá trình tối ưu hóa cục bộ (Appendix F.3).
    """
    # Tạo một ma trận unitary ngẫu nhiên để đại diện cho block đã học
    matrix = random_unitary(2**num_qubits)
    gate = UnitaryGate(matrix, label=label)
    return gate

def build_sewing_circuit():
    """
    Xây dựng mạch sử dụng kỹ thuật Sewing theo mô tả tại Fig. 3(b) và Appendix F.
    Mục tiêu: Thực hiện một phép biến đổi toàn cục bằng cách ghép các mảnh cục bộ
    sử dụng Ancilla để giảm độ sâu mạch.
    """
    
    # 1. Khởi tạo Register [cite: 340, 1147]
    # System qubits: Các qubit chứa dữ liệu chính
    # Ancilla qubits: Các qubit phụ trợ dùng để 'khâu' và song song hóa
    n_sys = 4  
    n_anc = 4  
    
    qr_sys = QuantumRegister(n_sys, 'sys')
    qr_anc = QuantumRegister(n_anc, 'anc')
    cr = ClassicalRegister(n_sys, 'output')
    
    qc = QuantumCircuit(qr_sys, qr_anc, cr)

    # --- GIAI ĐOẠN 1: DIVIDE (Áp dụng các mảnh lẻ) [cite: 343, 345] ---
    # Giả sử ta chia mạch lớn thành các mảnh nhỏ (U1, U3) hoạt động trên các cặp qubit rời rạc.
    # Điều này tương ứng với lớp đầu tiên trong Fig 3(b.3).
    
    # Mảnh 1 (U1): Tác động lên System qubit 0 và 1
    u1 = create_learned_piece(2, "U1_Learned")
    qc.append(u1, [qr_sys[0], qr_sys[1]])
    
    # Mảnh 2 (U3): Tác động lên System qubit 2 và 3
    u3 = create_learned_piece(2, "U3_Learned")
    qc.append(u3, [qr_sys[2], qr_sys[3]])
    
    # Lưu ý: Vì U1 và U3 rời rạc, chúng được thực thi song song (Depth = 1 block).
    qc.barrier()

    # --- GIAI ĐOẠN 2: SEWING (Khâu nối bằng SWAP) [cite: 266, 1150] ---
    # Sử dụng Ancilla để chuyển trạng thái, chuẩn bị cho việc áp dụng các mảnh chẵn (U2).
    # Kỹ thuật này giúp tránh việc phải chờ đợi (giảm độ sâu mạch) hoặc để sửa lỗi cục bộ.
    
    # Swap biên giới giữa các block để tạo liên kết cho lớp tiếp theo
    # Trong Fig 3(b.3), có bước "Swap w/ Anc".
    qc.swap(qr_sys[1], qr_anc[1])
    qc.swap(qr_sys[2], qr_anc[2])
    
    qc.barrier()

    # --- GIAI ĐOẠN 3: CONQUER (Áp dụng các mảnh chẵn đan xen) [cite: 291, 1148] ---
    # Áp dụng block U2 lên các qubit nằm ở biên giới giữa U1 và U3 cũ.
    # Nhờ Ancilla/Swap, ta có thể thực hiện việc này mà không làm tăng độ sâu 
    # quá mức trên các qubit hệ thống đang chờ.
    
    # Mảnh giữa (U2): Kết nối thông tin giữa block 1 và block 2 cũ
    # Ở đây ta tác động lên Ancilla[1] và Ancilla[2] (nơi đang giữ trạng thái của Sys[1], Sys[2])
    u2 = create_learned_piece(2, "U2_Learned")
    qc.append(u2, [qr_anc[1], qr_anc[2]])
    
    # (Tùy chọn) Swap lại về System nếu cần đo lường trên System
    qc.swap(qr_sys[1], qr_anc[1])
    qc.swap(qr_sys[2], qr_anc[2])

    # --- Đo lường kết quả ---
    qc.measure(qr_sys, cr)
    
    return qc

# --- Main Execution ---
if __name__ == "__main__":
    # Tạo mạch Sewing
    sewing_qc = build_sewing_circuit()
    
    # Vẽ mạch để hình dung cấu trúc "Divide-and-Conquer"
    print("Sơ đồ mạch Sewing (mô phỏng Fig 3b):")
    print(sewing_qc.draw(output='text')) # Dùng 'mpl' nếu chạy trên Jupyter Notebook
    
    # Chạy mô phỏng (cần cài qiskit-aer)
    try:
        from qiskit_aer import AerSimulator
        backend = AerSimulator()
        # Transpile mạch cho backend
        from qiskit import transpile
        t_qc = transpile(sewing_qc, backend)
        
        # Chạy và lấy kết quả
        result = backend.run(t_qc, shots=1024).result()
        counts = result.get_counts()
        print("\nKết quả đo lường (Counts):")
        print(counts)
        print("\nNote: Kết quả ngẫu nhiên vì các Unitary U1, U2, U3 được khởi tạo ngẫu nhiên.")
        
    except ImportError:
        print("\nCần cài đặt 'qiskit-aer' để chạy mô phỏng.")

Sơ đồ mạch Sewing (mô phỏng Fig 3b):
          ┌─────────────┐ ░        ░                ┌─┐               
   sys_0: ┤0            ├─░────────░────────────────┤M├───────────────
          │  U1_Learned │ ░        ░                └╥┘         ┌─┐   
   sys_1: ┤1            ├─░──X─────░─────────────────╫─────X────┤M├───
          ├─────────────┤ ░  │     ░                 ║     │    └╥┘┌─┐
   sys_2: ┤0            ├─░──┼──X──░─────────────────╫─────┼──X──╫─┤M├
          │  U3_Learned │ ░  │  │  ░                 ║ ┌─┐ │  │  ║ └╥┘
   sys_3: ┤1            ├─░──┼──┼──░─────────────────╫─┤M├─┼──┼──╫──╫─
          └─────────────┘ ░  │  │  ░                 ║ └╥┘ │  │  ║  ║ 
   anc_0: ────────────────░──┼──┼──░─────────────────╫──╫──┼──┼──╫──╫─
                          ░  │  │  ░ ┌─────────────┐ ║  ║  │  │  ║  ║ 
   anc_1: ────────────────░──X──┼──░─┤0            ├─╫──╫──X──┼──╫──╫─
                          ░     │  ░ │  U2_Learned │ ║  ║     │  ║  ║ 
   anc_2: ────────────────░─────X──░─┤1 

In [6]:
!pip install qiskit_algorithms

In [24]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, UnitaryGate
from qiskit.quantum_info import random_unitary
from qiskit_algorithms.optimizers import COBYLA
# SỬA LỖI: Dùng StatevectorSampler (V2) thay vì Sampler (V1)
from qiskit.primitives import StatevectorSampler 

# --- 1. SETUP BÀI TOÁN ---
NUM_QUBITS = 2
target_matrix = random_unitary(2**NUM_QUBITS).data
target_gate = UnitaryGate(target_matrix, label="Target_U")

print(f"Đang bắt đầu training khối {NUM_QUBITS}-qubit với Qiskit Primitives V2...")

# --- 2. XÂY DỰNG MODEL ---
ansatz = RealAmplitudes(num_qubits=NUM_QUBITS, reps=2, entanglement='linear')
num_params = ansatz.num_parameters

# --- 3. HÀM LOSS (CẬP NHẬT V2) ---
def cost_function(params):
    """
    Hàm tính loss sử dụng giao diện Primitives V2 mới.
    """
    qc = QuantumCircuit(NUM_QUBITS)
    
    # Target + Ansatz
    qc.append(target_gate, range(NUM_QUBITS))
    bound_ansatz = ansatz.assign_parameters(params)
    qc.append(bound_ansatz, range(NUM_QUBITS))
    
    # Quan trọng: measure_all() tạo ra classical register tên là "meas"
    qc.measure_all()
    
    # Khởi tạo Sampler V2
    sampler = StatevectorSampler()
    
    # Primitives V2 yêu cầu list các mạch (pubs)
    job = sampler.run([qc], shots=1024)
    
    # Lấy kết quả (PubResult)
    pub_result = job.result()[0]
    
    # TRUY XUẤT DỮ LIỆU KIỂU MỚI (Không dùng quasi_dists)
    # Dữ liệu nằm trong .data.<tên_register>.get_counts()
    # Vì dùng measure_all(), tên mặc định là 'meas'
    counts = pub_result.data.meas.get_counts()
    
    # Tính xác suất: Key bây giờ là BITSTRING ("00") chứ không phải số nguyên (0)
    target_state_str = "0" * NUM_QUBITS  # Ví dụ: "00"
    count_success = counts.get(target_state_str, 0)
    prob_success = count_success / 1024  # Chia tổng số shots
    
    return 1.0 - prob_success

# --- 4. TỐI ƯU HÓA ---
# Chạy tối ưu hóa như bình thường
optimizer = COBYLA(maxiter=2500)
initial_params = np.random.random(num_params) * 2 * np.pi

print("\nBắt đầu tối ưu hóa...")
# Lưu ý: Quá trình này có thể chậm hơn xíu do khởi tạo Sampler liên tục, 
# trong thực tế ta nên khởi tạo Sampler 1 lần bên ngoài nếu dùng Session, 
# nhưng với StatevectorSampler cục bộ thì không sao.
result = optimizer.minimize(fun=cost_function, x0=initial_params)

print(f"\nTraining hoàn tất!")
print(f"Loss cuối cùng: {result.fun:.5f}")

# --- 5. VALIDATION (CẬP NHẬT V2) ---
print("\n--- Kiểm tra hiệu quả (Validation) ---")
optimal_qc = QuantumCircuit(NUM_QUBITS)
optimal_qc.append(target_gate, range(NUM_QUBITS))
optimal_qc.append(ansatz.assign_parameters(result.x), range(NUM_QUBITS))
optimal_qc.measure_all()

sampler = StatevectorSampler()
job = sampler.run([optimal_qc], shots=2048)
pub_result = job.result()[0]
counts = pub_result.data.meas.get_counts()

print(f"Phân bố counts: {counts}")

target_state_str = "0" * NUM_QUBITS
success_prob = counts.get(target_state_str, 0) / 2048
print(f"Độ chính xác (Fidelity): {success_prob * 100:.2f}%")

if success_prob > 0.9:
    print("=> SUCCESS: Local Inversion thành công!")
else:
    print("=> FAIL: Cần điều chỉnh tham số hoặc tăng số lần lặp.")

Đang bắt đầu training khối 2-qubit với Qiskit Primitives V2...

Bắt đầu tối ưu hóa...

Training hoàn tất!
Loss cuối cùng: 0.15625

--- Kiểm tra hiệu quả (Validation) ---
Phân bố counts: {'00': 1653, '10': 152, '01': 91, '11': 152}
Độ chính xác (Fidelity): 80.71%
=> FAIL: Cần điều chỉnh tham số hoặc tăng số lần lặp.


/var/folders/lv/y3dprzzd05910718rydl2c7r0000gn/T/ipykernel_3449/2728878918.py:17: DeprecationWarning: The class ``qiskit.circuit.library.n_local.real_amplitudes.RealAmplitudes`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.real_amplitudes instead.
  ansatz = RealAmplitudes(num_qubits=NUM_QUBITS, reps=2, entanglement='linear')


In [25]:
!pip install "qiskit[aer]"

In [ ]:

import numpy as np
from math import sqrt
from collections import defaultdict
from scipy.optimize import minimize
from qiskit import QuantumCircuit, transpile
try:
    # Qiskit 1.x style Aer import
    from qiskit_aer import Aer
except Exception:
    # older Qiskit (pre-1.0)
    from qiskit import Aer

# -------------------------- Utility: bitstring helpers --------------------------

def reverse_bits(bitstr):
    return bitstr[::-1]

# -------------------------- Problem and hydraulic model --------------------------
# Scenario settings (edit these for your dam)
NUM_GATES = 5
# For each gate we specify the available discrete opening fractions (0..1)
# Example: 4 levels: [0.0, 0.33, 0.66, 1.0]
LEVELS_PER_GATE = [4] * NUM_GATES  # uniform levels; can be a list like [3,4,4,2,5]
BASE_LEVELS = [np.linspace(0.0, 1.0, L) for L in LEVELS_PER_GATE]

# Hydraulic coefficients per gate (k_i): flow (m^3/s) = k_i * opening_fraction * sqrt(max(0, h - h0))
K = np.array([12.0, 10.0, 8.0, 7.0, 6.0])[:NUM_GATES]
H0 = 1.0  # threshold head baseline

# Cost targets and penalties
ALPHA = 1.0  # weight to reach target T
BETA = 5.0   # overflow penalty weight
GAMMA_ONEHOT = 50.0  # penalty to enforce exactly one level per gate (should be large)
TARGET_T = 18.0
MAX_ALLOWED = 20.0

# QAOA / optimization settings
P = 2
SHOTS = 2000
QAOA_MAXITER = 80

# MPC settings
TIMESTEPS = 6
DT = 1.0  # time unit (arbitrary)
VOLUME_FACTOR = 1000.0  # converts net flow to change in head h (simplified)

# -------------------------- Build flow table per gate per level for a given head h --------------------------

def compute_flow_table_per_gate(head_h):
    """
    Returns flows_per_level: list of lists, flows_per_level[i][l] = flow of gate i when at level l (m^3/s)
    Using simple model: flow = k_i * opening_fraction * sqrt(max(0,h - h0))
    """
    flows = []
    effective = max(0.0, head_h - H0)
    sqrt_term = sqrt(effective) if effective > 0 else 0.0
    for i in range(NUM_GATES):
        k_i = float(K[i])
        levels = BASE_LEVELS[i]
        flows.append([k_i * level * sqrt_term for level in levels])
    return flows

# -------------------------- Build Hamiltonian coefficients from per-level flows --------------------------
# We use one-hot variables x_{i,l} mapped to Pauli Z via x = (1-Z)/2
# Let F = sum_{i,l} f_{i,l} x_{i,l}
# Cost = alpha*(F-T)^2 + beta*(F-M)^2 + gamma * sum_gate (sum_l x_{i,l} - 1)^2
# Expand to const + sum h_q Z_q + sum_{q<r} J_qr Z_q Z_r

class HamiltonianBuilder:
    def __init__(self, flows_per_level):
        # flows_per_level: list per gate of list per level
        self.flows_per_level = flows_per_level
        self.gates = len(flows_per_level)
        self.level_counts = [len(ls) for ls in flows_per_level]
        # map (gate, level) -> qubit index
        self.index_map = {}
        idx = 0
        for i in range(self.gates):
            for l in range(self.level_counts[i]):
                self.index_map[(i, l)] = idx
                idx += 1
        self.n_qubits = idx

    def build(self, alpha, beta, gamma_onehot, T, M):
        # initialize
        const = 0.0
        h = np.zeros(self.n_qubits)  # linear Z coeffs
        J = np.zeros((self.n_qubits, self.n_qubits))

        # S = sum_{i,l} f_{i,l} x_{i,l}; x = (1 - Z)/2
        # We will add quadratic term coeff_multiplier*(S - target)^2 twice (alpha and beta)
        def add_quadratic_S(coeff_multiplier, target_shift):
            nonlocal const, h, J
            # S = C0 + sum s_q Z_q where for each qubit q corresponding to (i,l):
            # x_q = (1 - Z_q)/2 => S = sum f_q * (1 - Z_q)/2 = C0 + sum s_q Z_q
            f_list = []
            for i in range(self.gates):
                for l in range(self.level_counts[i]):
                    f_list.append(self.flows_per_level[i][l])
            f = np.array(f_list)
            C0 = 0.5 * np.sum(f)
            s = -0.5 * f
            # (S - T)^2 = (C0 - T + sum s_q Z_q)^2
            const_term = coeff_multiplier * (C0 - target_shift) ** 2
            linear_term = 2.0 * coeff_multiplier * (C0 - target_shift) * s
            pair_const = coeff_multiplier * np.sum(s ** 2)
            pair_terms = 2.0 * coeff_multiplier * np.outer(s, s)
            const += const_term + pair_const
            h += linear_term
            for q in range(self.n_qubits):
                for r in range(q+1, self.n_qubits):
                    J[q, r] += pair_terms[q, r]

        add_quadratic_S(alpha, T)
        add_quadratic_S(beta, M)

        # One-hot penalty per gate: gamma * (sum_l x_{i,l} - 1)^2
        # For fixed gate i: let x_q be qubits for that gate. Expand same as above, but with x variables not Z
        for i in range(self.gates):
            # build vector f where for this gate x's coefficient = 1 for levels, others 0
            indices = [self.index_map[(i, l)] for l in range(self.level_counts[i])]
            # Let Sg = sum_{l in gate i} x_q. Use x_q = (1 - Z_q)/2 => Sg = C0g + sum s_q Z_q
            fvec = np.zeros(self.n_qubits)
            for idx in indices:
                fvec[idx] = 1.0
            C0g = 0.5 * np.sum(fvec)
            svec = -0.5 * fvec
            const_term = gamma_onehot * (C0g - 1.0) ** 2
            linear_term = 2.0 * gamma_onehot * (C0g - 1.0) * svec
            pair_const = gamma_onehot * np.sum(svec ** 2)
            pair_terms = 2.0 * gamma_onehot * np.outer(svec, svec)
            const += const_term + pair_const
            h += linear_term
            for q in range(self.n_qubits):
                for r in range(q+1, self.n_qubits):
                    J[q, r] += pair_terms[q, r]

        # Return const, h (size n_qubits), J (upper triangular meaningful)
        return const, h, J

# -------------------------- QAOA circuit builder using these Hamiltonian coefficients --------------------------

def cost_unitary_layer_from_hJ(qc, gamma, h, J):
    n = len(h)
    # linear terms -> RZ angle = 2 * gamma * h_i
    for i in range(n):
        ai = h[i]
        if abs(ai) < 1e-15:
            continue
        theta = 2.0 * gamma * ai
        qc.rz(theta, i)
    # pairwise terms: for i<j with J[i,j]
    for i in range(n):
        for j in range(i+1, n):
            jij = J[i, j]
            if abs(jij) < 1e-15:
                continue
            theta = 2.0 * gamma * jij
            qc.cx(i, j)
            qc.rz(theta, j)
            qc.cx(i, j)


def mixer_layer(qc, beta_param, n_qubits):
    for i in range(n_qubits):
        qc.rx(2.0 * beta_param, i)


def qaoa_circuit_from_hJ(params, p, h, J):
    gammas = params[:p]
    betas = params[p:2*p]
    n = len(h)
    qc = QuantumCircuit(n, n)
    for i in range(n):
        qc.h(i)
    for layer in range(p):
        cost_unitary_layer_from_hJ(qc, gammas[layer], h, J)
        mixer_layer(qc, betas[layer], n)
    qc.measure(range(n), range(n))
    return qc

# -------------------------- Expected cost (sampled) wrapper --------------------------
BACKEND = Aer.get_backend('qasm_simulator')

def expected_cost_sampled_for_hJ(params, p, h, J, const, shots=SHOTS):
    qc = qaoa_circuit_from_hJ(params, p, h, J)
    t_qc = transpile(qc, BACKEND)
    job = BACKEND.run(t_qc, shots=shots)
    counts = job.result().get_counts()
    total = 0.0
    tot_shots = 0
    for bitstr, c in counts.items():
        total += classical_cost_from_full_bitstring(bitstr, flows_per_level_global, const)
        tot_shots += c
    return total / tot_shots

# We need a function that computes classical cost from the FULL bitstring given current flows_per_level
# bitstring is in qiskit MSB..LSB ordering; map back to (gate,level)

flows_per_level_global = None  # will be set before optimizing
builder_global = None

def classical_cost_from_full_bitstring(bitstr, flows_per_level, const_term=0.0):
    # map bitstring to flows
    # reverse bitstr to match our index ordering (we used index 0..n-1 mapping)
    bs = bitstr[::-1]
    # compute S = sum f_q * x_q where x_q = 0/1 for bit
    total_flow = 0.0
    for (i, l), idx in builder_global.index_map.items():
        if bs[idx] == '1':
            total_flow += flows_per_level[i][l]
    # classical cost as defined earlier (alpha*|F-T| + beta*overflow) — used for reporting
    overflow = max(0.0, total_flow - MAX_ALLOWED)
    return ALPHA * abs(total_flow - TARGET_T) + BETA * overflow

# -------------------------- Optimization helper --------------------------

def optimize_qaoa_params(h, J, const, p=P, maxiter=QAOA_MAXITER):
    n_params = 2 * p
    x0 = 0.1 * np.random.randn(n_params)
    def obj(x):
        return expected_cost_sampled_for_hJ(x, p, h, J, const)
    res = minimize(fun=obj, x0=x0, method='COBYLA', options={'maxiter': maxiter})
    return res.x, res.fun

# -------------------------- Mapping optimized distribution -> chosen config --------------------------

def sample_and_pick_best(h, J, const, flows_per_level, params_opt, shots=5000):
    qc_opt = qaoa_circuit_from_hJ(params_opt, P, h, J)
    t_qc = transpile(qc_opt, BACKEND)
    job = BACKEND.run(t_qc, shots=shots)
    counts = job.result().get_counts()
    # evaluate classical cost for each observed bitstring and pick best
    best = None
    for bitstr, cnt in counts.items():
        c = classical_cost_from_full_bitstring(bitstr, flows_per_level, const)
        if best is None or c < best[0]:
            best = (c, bitstr, cnt)
    return best, counts

# -------------------------- Simple MPC loop --------------------------

def run_mpc(initial_head, inflow_forecast):
    """
    inflow_forecast: list of inflow rates per timestep (m^3/s)
    initial_head: initial water level (h)
    """
    h = initial_head
    history = []
    for t in range(TIMESTEPS):
        print(f"\n=== MPC step {t}, head={h:.3f} ===")
        # compute per-level flows for current head
        flows = compute_flow_table_per_gate(h)
        global flows_per_level_global, builder_global
        flows_per_level_global = flows
        builder = HamiltonianBuilder(flows)
        builder_global = builder
        const, h_vec, J_mat = builder.build(ALPHA, BETA, GAMMA_ONEHOT, TARGET_T, MAX_ALLOWED)
        print(f"Built Hamiltonian with n_qubits={builder.n_qubits}")
        # optimize QAOA parameters (this samples many times — costly). For demo, use small maxiter.
        params_opt, cost_opt = optimize_qaoa_params(h_vec, J_mat, const)
        print(f"Optimized params approx, expected cost={cost_opt:.4f}")
        best, counts = sample_and_pick_best(h_vec, J_mat, const, flows, params_opt)
        best_cost, best_bitstr, best_cnt = best
        print(f"Best sample: bitstr={best_bitstr}, cost={best_cost:.4f}, count={best_cnt}")
        # decode best bitstring into gate levels
        bs_rev = best_bitstr[::-1]
        chosen_levels = []
        for i in range(NUM_GATES):
            # find which level bit is 1 among the level_count
            lvl = None
            for l in range(builder.level_counts[i]):
                idx = builder.index_map[(i, l)]
                if bs_rev[idx] == '1':
                    lvl = l
                    break
            if lvl is None:
                # If none active, pick 0 (should be penalized by gamma)
                lvl = 0
            chosen_levels.append(lvl)
        # compute actual outflow
        total_outflow = sum(flows[i][chosen_levels[i]] for i in range(NUM_GATES))
        print(f"Chosen levels per gate: {chosen_levels}")
        print(f"Total outflow: {total_outflow:.3f} m^3/s")
        # update head with simple mass balance: h_next = h + (inflow - outflow) * dt / V
        inflow = inflow_forecast[t]
        net = inflow - total_outflow
        h = h + (net * DT) / VOLUME_FACTOR
        history.append({'t': t, 'head': h, 'chosen_levels': chosen_levels, 'outflow': total_outflow})
    return history

# -------------------------- Demo run (user can edit inflow forecast) --------------------------
if __name__ == '__main__':
    # Simple inflow forecast (m^3/s)
    inflow_forecast = [5.0, 6.0, 8.0, 12.0, 10.0, 7.0][:TIMESTEPS]
    h0 = 3.0
    print("Starting MPC demo (this will run QAOA multiple times; it may take time).")
    hist = run_mpc(h0, inflow_forecast)
    print("\nMPC run finished. History:")
    for entry in hist:
        print(entry)

# End of script


Starting MPC demo (this will run QAOA multiple times; it may take time).

=== MPC step 0, head=3.000 ===
Built Hamiltonian with n_qubits=20
Optimized params approx, expected cost=239.3549
Best sample: bitstr=00110100001100100001, cost=0.0866, count=1
Chosen levels per gate: [0, 1, 0, 2, 0]
Total outflow: 11.314 m^3/s

=== MPC step 1, head=2.994 ===
Built Hamiltonian with n_qubits=20
Optimized params approx, expected cost=241.1364
Best sample: bitstr=10100100000000000000, cost=0.1149, count=1
Chosen levels per gate: [0, 0, 0, 2, 1]
Total outflow: 9.413 m^3/s

=== MPC step 2, head=2.990 ===
Built Hamiltonian with n_qubits=20
Optimized params approx, expected cost=219.3566
Best sample: bitstr=01110000001000000010, cost=0.1302, count=1
Chosen levels per gate: [1, 0, 1, 0, 0]
Total outflow: 9.405 m^3/s

=== MPC step 3, head=2.989 ===
Built Hamiltonian with n_qubits=20
Optimized params approx, expected cost=241.7674
Best sample: bitstr=00000001001010000000, cost=0.1366, count=1
Chosen levels